In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'
torch.cuda.set_device(I_GPU)

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

from torch_points3d.visualization.multimodal_data import visualize_mm_data

In [ ]:
# import plotly.offline as pyo
# pyo.init_notebook_mode()

import plotly.io as pio
# pio.renderers

# For local jupyterlab
pio.renderers.default = 'jupyterlab'

# Ror remote HPC jupyterhub.
# Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'
# pio.renderers.default = 'iframe_connected'  

# Helpers

In [ ]:
from torch_geometric.data import Data, Batch
from torch import Tensor

OBJECT_COLOR = [
        [233, 229, 107],  #'ceiling' .-> .yellow
        [95, 156, 196],  #'floor' .-> . blue
        [179, 116, 81],  #'wall'  ->  brown
        [241, 149, 131],  #'beam'  ->  salmon
        [81, 163, 148],  #'column'  ->  bluegreen
        [77, 174, 84],  #'window'  ->  bright green
        [108, 135, 75],  #'door'   ->  dark green
        [41, 49, 101],  #'chair'  ->  darkblue
        [79, 79, 76],  #'table'  ->  dark grey
        [223, 52, 52],  #'bookcase'  ->  red
        [89, 47, 95],  #'sofa'  ->  purple
        [81, 109, 114],  #'board'   ->  grey
        [233, 233, 229],  #'clutter'  ->  light grey
        [0, 0, 0],  # unlabelled .->. black
    ]

CLASSES = [
    'ceiling',
    'floor',
    'wall',
    'beam',
    'column',
    'window',
    'door',
    'chair',
    'table',
    'bookcase',
    'sofa',
    'board',
    'clutter',
    'unlabelled',
]

In [ ]:
from torch_geometric.transforms import Center, RandomRotate
from torch_points3d.core.data_transform import RandomNoise, RandomScaleAnisotropic, RandomSymmetry, \
    DropFeature, XYZFeature, AddFeatsByKeys
from torch_points3d.core.data_transform.multimodal.image import ToFloatImage, AddPixelHeightFeature, \
    PickImagesFromMappingArea, PickImagesFromMemoryCredit, CropImageGroups, Normalize
from torch_points3d.datasets.base_dataset import BaseDataset
from torch_points3d.datasets.base_dataset_multimodal import BaseDatasetMM

def sample_real_data(tg_dataset, idx=0):
    """
    Temporarily remove the 3D and 2D transforms affecting the point 
    positions and images from the dataset to better visualize points 
    and images relative positions.
    """
    transform = tg_dataset.transform
    tg_dataset.transform = BaseDataset.remove_transform(transform, [Center, RandomNoise,
        RandomRotate, RandomScaleAnisotropic, RandomSymmetry, DropFeature, AddFeatsByKeys])
    
    transform_image = tg_dataset.transform_image
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, AddPixelHeightFeature,])
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, PickImagesFromMappingArea, AddPixelHeightFeature, ])
#     tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, PickImagesFromMappingArea, AddPixelHeightFeature,PickImagesFromMemoryCredit ])
    #tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, AddPixelHeightFeature,PickImagesFromMemoryCredit ])
    tg_dataset.transform_image = BaseDatasetMM.remove_multimodal_transform(transform_image, [ToFloatImage, AddPixelHeightFeature, Normalize ])

    out = tg_dataset[idx]
    
    tg_dataset.transform = transform
    tg_dataset.transform_image = transform_image
    return out

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
# DATA_ROOT = '/raid/datasets/pointcloud/data/s3dis'  # ENGIE

overrides = [
    'task=segmentation',
#     'data=segmentation/multimodal/s3disfused/no3d_5cm_768x384-exact',
    'data=segmentation/multimodal/s3disfused-sparse',
    'models=segmentation/multimodal/sparseconv3d',
    'model_name=Res16UNet34-L4-early-ade20k-interpolate',
    'data.fold=5',
    'data.sample_per_epoch=2000',
#     f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_1024x512')}",
    f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_512x256')}",
    
#     f"data.dataroot={os.path.join(DATA_ROOT, 'temp')}",
#     'data.resolution_2d=[512, 256]'
]

cfg = hydra_read(overrides)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

# Load S3DIS dataset

In [ ]:
from time import time
start = time()

dataset = S3DISFusedDataset(cfg.data)
# print(dataset)

print(f"Time = {time() - start:0.1f} sec.")

# Visualize multimodal spherical samples

In [ ]:
# Remove pixel memory credit transform
from torch_points3d.core.data_transform.multimodal.image import PickImagesFromMemoryCredit
from torch_points3d.datasets.base_dataset_multimodal import BaseDatasetMM
for x in [dataset.train_dataset, dataset.val_dataset, dataset.test_dataset[0]]:
    x.transform_image = BaseDatasetMM.remove_multimodal_transform(x.transform_image, [PickImagesFromMemoryCredit])

In [ ]:
# Get multimodal spherical sample
# mm_data = sample_real_data(dataset.test_dataset[0], idx=100)
# mm_data = sample_real_data(dataset.train_dataset)
# mm_data = sample_real_data(dataset.val_dataset)

idx = 215
mm_data = sample_real_data(dataset.test_dataset[0], idx=idx)
# print(mm_data)

# credit_used = sum([np.prod(im.img_size) * im.num_views for im in mm_data.modalities['image']])
# credit_max = dataset.train_transform_image.transforms[-1].credit
# print(f"Used pixel credit: {credit_used} / {credit_max} = {credit_used / credit_max * 100:0.1f} %")

In [ ]:
visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, front='y', alpha=3, pointsize=5)

In [ ]:
from torch_points3d.core.multimodal.image import ImageData

idx = 215

# Set the minimum crop size
dataset.test_dataset[0].transform_image.transforms[-2].min_size = 128

# Sample
mm_data = sample_real_data(dataset.test_dataset[0], idx=idx)

# Select the points
idx_subsphere = torch.where(mm_data.data.pos[:, 1] < -7.6)[0]
mm_data_clean = mm_data[idx_subsphere]

# Select the images
# mm_data_clean.modalities['image'] = ImageData([
#     mm_data_clean.modalities['image'][2][1],
#     mm_data_clean.modalities['image'][1],
#     mm_data_clean.modalities['image'][-2][0],
#     mm_data_clean.modalities['image'][-1]
# ])
mm_data_clean.modalities['image'] = ImageData([
    mm_data_clean.modalities['image'][0][1],
    mm_data_clean.modalities['image'][1][1],
    mm_data_clean.modalities['image'][2]
])

# Set all but object of interest to label 0
# mask_object = mm_data_clean.data.y == 11  # board
# mask_object = torch.logical_and(mm_data_clean.data.pos[:, 1] > -9.3, mm_data_clean.data.y == 7)  # chair in the middle

# # a = torch.where(mask_object)[0]
# # a[torch.randint(low=0, high=a.shape[0], size=(1,))]
# # mask_object = torch.zeros_like(mask_object)
# # mask_object[a[torch.randint(low=0, high=a.shape[0], size=(1,))]] = True

# mm_data_clean.y[~mask_object] = 0

mm_data_clean.y[mm_data_clean.y == 0] = 8

# # Keep only mappings to the object of interest
# idx_object = torch.where(mask_object)[0]
# for im in mm_data_clean.modalities['image']:
#     im.mappings = im.mappings[idx_object]
#     im.mappings = im.mappings.reindex_groups(idx_object, num_groups=mask_object.shape[0])

visualize_mm_data(mm_data_clean, class_names=CLASSES, class_colors=OBJECT_COLOR, width=1000, height=750, voxel=0.05, show_3d=True, show_2d=False, color_mode='ligth', alpha=2, pointsize=6)

In [ ]:
# # Downscale 3D
# from torch_points3d.core.data_transform import GridSampling3D
# from torch_points3d.core.data_transform.multimodal.image import SelectMappingFromPointId

# print(mm_data.data.mapping_index)
# mm_data.data = GridSampling3D(0.2)(mm_data.data.clone())
# print(mm_data.data.mapping_index)

# # Subsample the mappings accordingly
# mm_data.data , mm_data.images = SelectMappingFromPointId()(mm_data.data, mm_data.images)

In [ ]:
# # Downscale 3D
# from torch_points3d.core.data_transform import GridSampling3D
# from torch_points3d.core.data_transform.multimodal.image import SelectMappingFromPointId

# print(mm_data.data.mapping_index)
# mm_data.data = GridSampling3D(0.2)(mm_data.data.clone())
# print(mm_data.data.mapping_index)

# # Subsample the mappings accordingly
# mm_data.data , mm_data.images = SelectMappingFromPointId()(mm_data.data, mm_data.images)

In [ ]:
# # Downscale 3D with 'merge' mode
# from torch_points3d.core.data_transform import GridSampling3D
# from torch_cluster import grid_cluster
# from torch_geometric.nn import voxel_grid
# from torch_geometric.nn.pool.consecutive import consecutive_cluster

# voxel = 0.1

# # Get the cluster indices for grid sampling
# coords = torch.round((mm_data.data.pos) / voxel)
# if "batch" not in mm_data.data:
#     cluster = grid_cluster(coords, torch.tensor([1, 1, 1]))
# else:
#     cluster = voxel_grid(coords, mm_data.data.batch, 1)
# cluster, _ = consecutive_cluster(cluster)

# # Actual 3D sampling
# mm_data.data = GridSampling3D(voxel)(mm_data.data.clone())

# # Subsample the mappings accordingly with 'merge' mocde
# for im in mm_data.images:
#     im.mappings = im.mappings.select_points(cluster, merge=True)
    
# # Reset the mapping_index
# # Remark : unlike SelectMappingFromPointId, we don't need to search for
# # potentially unseen images when 'merge=True', because the subsampling
# # index implies that all points are merged into a new one, so no 
# # mappings should be lost in the process.
# mm_data.data.mapping_index = torch.arange(mm_data.data.num_nodes)

In [ ]:
# # Downscale 2D
# for im in mm_data.images:
#     im = im.update_features_and_scale(im.images[:, :, ::4, ::4])

In [ ]:
# # Multimodal batch
# from torch_points3d.datasets.multimodal.data import MMBatch
# mm_data = MMBatch.from_mm_data_list([sample_real_data(dataset.test_dataset[0], idx=i) for i in [0, 2]])

In [ ]:
# visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='y', alpha=3)

In [ ]:
# # # Export to HTML for each image coloring mode
# for mode in ['light', 'rgb', 'pos', 'y']:
#     visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, figsize=1000, voxel=0.1, show_3d=True, 
#         show_2d=True, alpha=3, color_mode=mode, path=f'/home/ign.fr/drobert-admin/Bureau/s3dis_visualizations/demo_mm_data/sub_2d__{mode}.html',
#         title=f'2D downsampled multimodal sample - Mode={mode}')

# Visualize whole areas

In [ ]:
from torch_points3d.datasets.multimodal.data import MMData

# mm_data_large = MMData(dataset.train_dataset._datas[1], dataset.train_dataset._images[1], dataset.train_dataset._mappings[1])
mm_data_large = MMData(dataset.test_dataset[0]._datas[0], dataset.test_dataset[0]._images[0], dataset.test_dataset[0]._mappings[0])
visualize_mm_data(mm_data_large, class_names=CLASSES, class_colors=OBJECT_COLOR, figsize=1600, voxel=0.05, show_2d=False, max_points=500000)

# Test multimodal batching

In [ ]:
from torch_points3d.datasets.multimodal.data import MMData, MMBatch 

# Get multimodal spherical sample
# mm_data = sample_real_data(dataset.test_dataset[0], idx=50)
mm_data_1 = sample_real_data(dataset.train_dataset)
mm_data_2 = sample_real_data(dataset.train_dataset)
mm_data_3 = sample_real_data(dataset.train_dataset)

mm_data_1.images.num_views, mm_data_2.images.num_views, mm_data_3.images.num_views

In [ ]:
# removing masks to bypass a local version bug
mm_data_1.images.mask = mm_data_2.images.mask = mm_data_3.images.mask = None

In [ ]:
mm_data_1

In [ ]:
# Visualize batch
mm_batch = MMBatch.from_mm_data_list([mm_data_1, mm_data_2, mm_data_3])
image_batch = mm_batch.images.read_images(size=mm_batch.images.map_size_low)
visualize_mm_data(mm_batch, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch, figsize=800, voxel=0.05, show_3d=True, show_2d=True, color_mode='y')

In [ ]:
# Visualize sampling #3
image_batch_3 = mm_data_3.images.read_images(size=mm_data_3.images.map_size_low)
visualize_mm_data(mm_data_3, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch_3, figsize=800, voxel=0.05, show_3d=True, show_2d=True, color_mode='y')

In [ ]:
# Visualize sampling #3 after batch unpacking
mm_data_unpacked_1, mm_data_unpacked_2, mm_data_unpacked_3 = mm_batch.to_mm_data_list()
image_batch_3 = mm_data_unpacked_3.images.read_images(size=mm_data_unpacked_3.images.map_size_low)
visualize_mm_data(mm_data_unpacked_3, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch_3, figsize=800, voxel=0.05, show_3d=True, show_2d=True, color_mode='y')

In [ ]:
print(np.array_equal(mm_data_3.mappings.values[1].values[0], mm_data_unpacked_3.mappings.values[1].values[0]))

# Visualize 3D resampling of multimodal data
Under the hood, this calls the indexing mechanism of mappings

Remark: this does not make use of `ImageMapping.select_points`. If we wanted to test the `merge` mechanism of `select_points`, we would need to work with a remapping of the points, much like a voxel sampling. 

In [ ]:
# Get multimodal spherical sample
# mm_data = sample_real_data(dataset.test_dataset[0], idx=50)
mm_data = sample_real_data(dataset.train_dataset)
print(mm_data)

In [ ]:
# Load images
image_batch = mm_data.images.read_images(size=mm_data.images.map_size_low)

# Visualize
visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='y')

In [ ]:
# Slice the multimodal data with random indices
idx = torch.randint(low=0, high=mm_data.data.num_nodes, size=(int(mm_data.data.num_nodes/4),))
mm_data_sliced = mm_data[idx]

# Load images (could have been skipped had we used MMData.load)
image_batch_sliced = mm_data_sliced.images.read_images(size=mm_data_sliced.images.map_size_low)

# Visualize
visualize_mm_data(mm_data_sliced, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch_sliced, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='y')

# Visualize 2D resampling of multimodal data

In [ ]:
# Get multimodal spherical sample
# mm_data = sample_real_data(dataset.test_dataset[0], idx=50)
mm_data = sample_real_data(dataset.train_dataset)
print(mm_data)

In [ ]:
# Load images
image_batch = mm_data.images.read_images(size=mm_data.images.map_size_low)

# Visualize
visualize_mm_data(mm_data, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='pos')

In [ ]:
from torch_points3d.datasets.multimodal.data import MMData
import copy

# Change image resolution
ratio = 2
map_size_low_resized = tuple([int(x / ratio)for x in mm_data.images.map_size_low])
image_batch_resized = mm_data.images.read_images(size=map_size_low_resized)

# Change mapping 2D resolution
mm_data_resized = copy.deepcopy(mm_data)  # use clone instead
mm_data_resized.mappings = mm_data_resized.mappings.downscale_views(ratio)

# Visualize
visualize_mm_data(mm_data_resized, class_names=CLASSES, class_colors=OBJECT_COLOR, image_batch=image_batch_resized, figsize=1000, voxel=0.05, show_3d=True, show_2d=True, color_mode='pos')

# Visualize visibility models

In [ ]:
from torch_points3d.core.multimodal.visibility import visibility
from torch_points3d.datasets.segmentation.multimodal.s3dis import s3dis_image_area
from torch_points3d.core.multimodal.visibility import read_s3dis_depth_map
import matplotlib.pyplot as plt
from PIL import Image

data = dataset.test_dataset[0]._datas[0]
images = dataset.test_dataset[0]._images[0]

img_selection = np.random.choice(np.arange(images.num_views), 5, replace=False)
for i_image, im in zip(img_selection, images[img_selection]):
    # img_size = (512, 256)
    img_size = (256, 158)
    # img_size = im.img_size

    xyz_to_img = (data.pos - im.pos.squeeze()).float()
    img_opk = im.opk.squeeze().float()
    linearity = data.linearity
    planarity = data.planarity
    scattering = data.scattering
    normals = data.norm
    img_mask = torch.from_numpy(np.array(Image.fromarray(im.mask.t().numpy()).resize(img_size))).t()
    
    depth_threshold=0.05
    depth_map_path = os.path.join(
        DATA_ROOT, 'shared/image', s3dis_image_area(im.path[0]), 'pano/depth', 
        os.path.basename(im.path[0]).replace('_rgb.png', '_depth.png'))
    use_cuda=True
    
    # Recover the real RGB and depth images
    real_rgb = np.array(Image.open(os.path.join(DATA_ROOT, *im.path[0].split('/')[-7:])).resize((2*img_size[0], 2*img_size[1])))
    real_depth_map = read_s3dis_depth_map(path, img_size=img_size, empty=-1)
    real_depth_map[real_depth_map < im.r_min] = -1
    real_depth_map[real_depth_map > im.r_max] = -1
    real_depth_map[torch.from_numpy(np.logical_not(img_mask.numpy()))] = -1
    
    # Plot the RGB and depth images
    fig, axes = plt.subplots(int((len(methods) + 2)/2), 2, figsize=(12*2, 8 * int((len(methods) + 2)/2)))
    fig.suptitle(f'S3DIS Area 5 image {i_image}', fontsize=24)
    axes[0, 0].imshow(real_rgb)
    axes[0, 0].set_title('RGB')
    depth_map_kwargs = {'cmap':'nipy_spectral', 'vmin':-im.r_max, 'vmax':-im.r_min, 'interpolation': 'nearest'}
    axes[0, 1].imshow(-real_depth_map.t().numpy(), **depth_map_kwargs)
    axes[0, 1].set_title('True Depth')
    
    methods = ['splatting', 'splatting_exact', 'depth_map', 'biasutti']
    for i_method, method in enumerate(methods):
                
        out = visibility(
            xyz_to_img, img_opk, linearity=linearity, planarity=planarity,
            scattering=scattering, normals=normals, img_mask=img_mask,
            img_size=img_size, voxel=im.voxel, r_max=im.r_max,
            r_min=im.r_min, k_swell=im.growth_k, d_swell=im.growth_r,
            exact='exact' in method, method=method.replace('_exact', ''), depth_threshold=depth_threshold, 
            depth_map_path=depth_map_path, use_cuda=use_cuda, biasutti_k=75,
            biasutti_margin=64, biasutti_threshold=0.5)
    
        # proj_map = torch.zeros(img_size).long()
        # proj_map[out['x'].long().cpu(), out['y'].long().cpu()] = out['idx'].cpu()

        # proj_map = torch.zeros(img_size).float()
        # proj_map[out['x'].long().cpu(), out['y'].long().cpu()] = out['features'][:, 0].cpu()

        proj_map = torch.zeros(img_size).float()
        proj_map[out['x'].long().cpu(), out['y'].long().cpu()] = out['depth'].cpu()
        
        # Plot the computed map
        pcent_mapped_points = round(100 * out['idx'].shape[0] / xyz_to_img.shape[0], 1)
        pcent_mapped_pixels = round(100 * n_mappings / (img_size[0] * img_size[1]), 1)
        ax = axes[(i_method+2) // 2, (i_method+2) % 2]
        ax.imshow(-proj_map.t().numpy(), **depth_map_kwargs)
        ax.set_title(f'Visibility - {method} - {pcent_mapped_points}% points mapped - {pcent_mapped_pixels}% pixels mapped')
   
    plt.show()
    print('\n\n')

# S3DIS annotation errors

### Room orientation errors
- Area 1: None
- Area 2: hallway_11 (+180°) 
- Area 3: None
- Area 4: None
- Area 5: hallway 6 (or 7?) (+180°)
- Area 6: None

### Annotation errors
- Area 5: office_27 - ceiling points a bit everywhere